# QT User Meeting Tutorial 3
## Pulse level sequencing

Copyright (C) 2022 Zurich Instruments

This software may be modified and distributed under the terms of the MIT license. See the LICENSE file for details.

# Import packages

In [1]:
# This notebook uses the Toolkit, which builds on our Python API
from zhinst.toolkit import Session
from zhinst.toolkit import CommandTable
from zhinst.toolkit import Waveforms
import numpy as np

# Define constants

In [2]:
## These are all the settings used throughout the entire notebook.

# Connection configuration:
device_id             = 'dev12127'
server_host           = "127.0.0.1"

# Analog RF output:
channel_index         = 0 #0...7
output_range          = -5 #dBm
center_frequency      = 1.5e9 #Hz
rflf_path             = 1 #RF path
enable                = 1
disable               = 0

# Digital modulation:
osc1_frequency        = -125.521e6 #Hz
osc2_frequency        = 339.197e6 #Hz
osc1_index            = 0 #0...7
osc2_index            = 1 #0...7
phase                 = 25.2 #degrees
global_amp            = 0.5 #0...1
gains_cw              = (0.0, 0.95, 0.95, 0.0) #unitless
gains_mod             = (1.0, -1.0, 1.0, 1.0) #unitless

# Markers and triggers
trigger_in_source     = 'inttrig'
trigger_in_slope      = 'rising_edge'
marker_out_source     = 'output0_marker0'
repetitions           = 1e9 #unitless
holdoff               = 2.5e-6 #s

## Sequencer:
# General:
single                = 0 #repeat sequence
samp_rate             = 2e9 # Gb/s
amp                   = 1.0 #unitless, 0...1
length                = 50e-9 #s
readout               = 0.5e-6 #s
n_aves                = 10000

# Ramsey
n_evos                = 100 #number of evolution times for Ramsey
evo_step              = 16e-9 #s

# Phase-sweep Ramsey
n_phases              = 100 #number of phases for Ramsey

# Sub-sample Ramsey
subsampling_bits      = 2
t_start               = 16 #samples
t_stop                = 5000 #samples
t_step                = 1 #samples

# Rabi
n_amps                = 100 #number of amplitudes for Rabi

# DRAG calibration
q_start               = -1.0 #first qscale value
q_stop                = 1.0 #last qscale value
n_qscale              = 100 #number of qscale values
q_step                = (q_stop - q_start) / n_qscale #qscale step size

# Randomized benchmarking
pi_amplitude          = 1.0
pi2_amplitude         = 0.5
pi_length             = length
pi2_length            = length
awg_register_m1       = 0
awg_register_seed     = 1
awg_register_recovery = 2

# Connect to device

In [3]:
session = Session(server_host)
device = session.connect_device(device_id)
#device.check_compatibility()

# Generate a CW Signal

In [4]:
# Configure RF output
device.sgchannels[channel_index].configure_channel(enable = enable, output_range = output_range, center_frequency = center_frequency, rf_path = rflf_path)

# Configure digital sine generator
device.sgchannels[channel_index].configure_sine_generation(enable = enable, osc_index = osc1_index, osc_frequency = osc1_frequency, phase = phase, gains = gains_cw)

# Generate a Gaussian pulse

In [6]:
# Disable cw signal generation
device.sgchannels[channel_index].sines[0].I.enable(disable)
device.sgchannels[channel_index].sines[0].Q.enable(disable)

# Configure digital modulation
device.sgchannels[channel_index].configure_pulse_modulation(enable = enable, osc_index = osc1_index, osc_frequency = osc1_frequency, phase = phase, global_amp = global_amp, gains = gains_mod)

# Configure marker outputs and trigger inputs of the sequencer
device.sgchannels[channel_index].awg.configure_marker_and_trigger(trigger_in_source = trigger_in_source, trigger_in_slope = trigger_in_slope, marker_out_source = marker_out_source)

# Configure internal trigger unit
with device.set_transaction():
    device.system.internaltrigger.enable(disable) #turn off in case it was running before
    device.system.internaltrigger.repetitions(repetitions)
    device.system.internaltrigger.holdoff(holdoff)


In [7]:
seqc_str = f"""
//Define constants
const amp = {amp};
const length = ceil(DEVICE_SAMPLE_RATE*{length}/32)*32; //convert length in time to samples
const width = length/8;

//Waveform definition
wave wI = gauss(length,amp,length/2,width);
wave m = marker(length,1);
wave wIm = wI + m; //combine waveform and marker data

repeat ({repetitions}) {{
  waitDigTrigger(1); //wait for trigger from internal trigger unit
  resetOscPhase();
  playWave(1,2,wIm);
}}
"""

In [8]:
# Upload sequence and enable sequencer
device.sgchannels[channel_index].awg.load_sequencer_program(seqc_str)
device.sgchannels[channel_index].awg.enable_sequencer(single = single)

In [9]:
#Start internal trigger
device.system.internaltrigger.enable(enable)

# Ramsey Sequence

In [1]:
seqc_ramsey = f"""
//Define constants
const num_evos = {n_evos};
const evo_step = ceil(DEVICE_SAMPLE_RATE*{evo_step}/16)*16;
const num_aves = {n_aves};
const amp = {amp};
const length = ceil(DEVICE_SAMPLE_RATE*{length}/16)*16;
const readout = ceil(DEVICE_SAMPLE_RATE*{readout}/16)*16;
const width = length/8;

//Waveform definition
wave wI = gauss(length,amp,length/2,width);
wave wQ = drag(length,amp,length/2,width);
wave m = marker(readout,1);

//Assign index and outputs
assignWaveIndex(1,2,wI,1,2,wQ,0);

//Execution, sequential averaging
do {{
  var i;
  for (i = 0; i < num_evos; i++) {{           //Evolution time loop
    repeat (num_aves) {{                      //Averaging loop
      waitDigTrigger(1);
      resetOscPhase();                        //Reset the oscillator

      executeTableEntry(0);                   //First pi/2 pulse
      playZero(32+i*evo_step);                //Evolution time
      executeTableEntry(0);                   //Second pi/2 pulse
      
      playWave(m);                            //Readout
      waitWave();
    }}
  }}
}} while (1);
"""

NameError: name 'n_evos' is not defined

In [11]:
# Disable internal trigger
device.system.internaltrigger.enable(disable)

# Define command table
ct_schema = device.sgchannels[channel_index].awg.commandtable.load_validation_schema()
ct_ramsey = CommandTable(ct_schema)
ct_ramsey.table[0].waveform.index = 0 # Define table entry 0 to use waveform index 0

# Upload sequence
device.sgchannels[channel_index].awg.load_sequencer_program(seqc_ramsey)

# Upload command table
device.sgchannels[channel_index].awg.commandtable.upload_to_device(ct_ramsey)

# Enable sequencer
device.sgchannels[channel_index].awg.enable_sequencer(single = single)

In [13]:
#Start internal trigger
device.system.internaltrigger.enable(enable)

# Ramsey with Sub-Sample Resolution

In [27]:
seqc_ramsey = f"""
//Define constants
const num_evos = {n_evos};
const evo_step = ceil(DEVICE_SAMPLE_RATE*{evo_step}/32)*32;
const num_aves = {n_aves};
const amp = {amp};
const length = ceil(DEVICE_SAMPLE_RATE*{length}/32)*32;
const readout = ceil(DEVICE_SAMPLE_RATE*{readout}/32)*32;
const width = length/8;

const subsampling = pow(2,{subsampling_bits});
const all_bits = {subsampling_bits} + 4;

const mask_fine = pow(2,all_bits) - 1;
const mask_coarse = -pow(2,all_bits);

//Waveform definition
wave wI0 = gauss(length, amp, length/2, width);
wave wQ0 = drag(length, amp, length/2, width);
wave m = marker(readout,1);

//Create shifted waveforms
cvar j,k;
for (j = 0; j < 16; j++) {{
  for (k = 0; k < subsampling; k++) {{
    wave wI = gauss(length, amp, length/2 + k/subsampling, width);
    wave wQ = drag(length, amp, length/2 + k/subsampling, width);
    wave wI_shifted = join(zeros(j), wI, zeros(16-j));             //Create the j-samples shifted waveform
    wave wQ_shifted = join(zeros(j), wQ, zeros(16-j));             //Create the j-samples shifted waveform
    assignWaveIndex(1,2,wI_shifted,1,2,wQ_shifted,subsampling*j+k);   //Assign index to the waveform
  }}
}}

do {{
  //Execution, sequential averaging
  var t = {t_start};
  var t_fine, coarse, t_coarse;
  do {{
      t_fine = t & mask_fine;                   //The fine shift is the four least significant bits
      coarse = t & mask_coarse;                 //Check if coarse delay is needed (boolean). Equivalent to (t >= 16)
      t_coarse = t>>{subsampling_bits};
      
      waitDigTrigger(1);
      resetOscPhase();                          //Reset the oscillator

      playWave(1,2,wI0,1,2,wQ0);                //Play first pulse, no shift
      if(coarse)
        playZero(t_coarse);                     //Evolution time t (coarse)
      executeTableEntry(t_fine);                //Play second pulse, fine shift
      
      playWave(m);                              //Readout trigger
      waitWave();
      
      t += {t_step};                            //Increase wait time
  }} while (t < {t_stop});                      //Loop until the end
}} while(1);
"""

In [28]:
# Disable internal trigger
device.system.internaltrigger.enable(disable)

# Define command table
ct_ramsey.clear()
ct_schema = device.sgchannels[channel_index].awg.commandtable.load_validation_schema()
ct_ramsey = CommandTable(ct_schema)
for i in range(16*2**subsampling_bits):
    ct_ramsey.table[i].waveform.index = i

# Upload sequence
device.sgchannels[channel_index].awg.load_sequencer_program(seqc_ramsey)

# Upload command table
device.sgchannels[channel_index].awg.commandtable.upload_to_device(ct_ramsey)

# Enable sequencer
device.sgchannels[channel_index].awg.enable_sequencer(single = single)

In [29]:
device.system.internaltrigger.enable(enable)

# Phase-sweep Ramsey

In [129]:
seqc_ramsey = f"""
//Define constants
const num_phases = {n_phases};
const num_aves = {n_aves};
const amp = {amp};
const length = ceil(DEVICE_SAMPLE_RATE*{length}/32)*32;
const readout = ceil(DEVICE_SAMPLE_RATE*{readout}/32)*32;
const width = length/8;

//Waveform definition
wave wI = gauss(length,amp,length/2,width);
wave wQ = drag(length,amp,length/2,width);
wave m = marker(readout,1);

//Assign index and outputs
assignWaveIndex(1,2,wI,1,2,wQ,0);

//Execution, sequential averaging
do {{
  var phase = 0;
  do {{                                        //Evolution time loop
    repeat (num_aves) {{                       //Averaging loop
      waitDigTrigger(1);
      resetOscPhase();                         //Reset the oscillator
      
      executeTableEntry(num_phases);           //First pi/2 pulse
      executeTableEntry(phase);                //Second pi/2 pulse
      
      playWave(m);                             //Readout trigger
      waitWave();
    }}
    phase += 1;
  }} while (phase < num_phases);
}} while (1);
"""

In [130]:
# Disable internal trigger
device.system.internaltrigger.enable(disable)

# Define command table
ct_ramsey.clear()
ct_schema = device.sgchannels[channel_index].awg.commandtable.load_validation_schema()
ct_ramsey = CommandTable(ct_schema)
for i in range(n_phases):
    ct_ramsey.table[i].waveform.index = 0
    ct_ramsey.table[i].phase.value = 360*i/n_phases
    ct_ramsey.table[i].phase.increment = False
    ct_ramsey.table[i].amplitude00.value = 1.0
    ct_ramsey.table[i].amplitude00.increment = False
    ct_ramsey.table[i].amplitude01.value = -1.0
    ct_ramsey.table[i].amplitude01.increment = False
    ct_ramsey.table[i].amplitude10.value = 1.0
    ct_ramsey.table[i].amplitude10.increment = False
    ct_ramsey.table[i].amplitude11.value = 1.0
    ct_ramsey.table[i].amplitude11.increment = False
ct_ramsey.table[n_phases].waveform.index = 0
ct_ramsey.table[n_phases].phase.value = 0
ct_ramsey.table[n_phases].phase.increment = False

In [131]:
# Upload sequence
device.sgchannels[channel_index].awg.load_sequencer_program(seqc_ramsey)

# Upload command table
device.sgchannels[channel_index].awg.commandtable.upload_to_device(ct_ramsey)

# Enable sequencer
device.sgchannels[channel_index].awg.enable_sequencer(single = single)

In [132]:
device.system.internaltrigger.enable(enable)

# Rabi sequence

In [133]:
seqc_rabi = f"""
//Define constants
const num_amps = {n_amps};
const num_aves = {n_aves};
const amp = {amp};
const length = ceil(DEVICE_SAMPLE_RATE*{length}/32)*32;
const readout = ceil(DEVICE_SAMPLE_RATE*{readout}/32)*32;
const width = length/8;

//Waveform definition
wave wI = gauss(length,amp,length/2,width);
wave wQ = drag(length,amp,length/2,width);
wave m = marker(readout,1);

//Assign index and outputs
assignWaveIndex(1,2,wI,1,2,wQ,0);

//Execution, sequential averaging
do {{
  executeTableEntry(0);                   //Set start amplitude
  repeat (num_amps-1) {{                  //Amplitude loop
    repeat (num_aves) {{                  //Averaging loop      
      waitDigTrigger(1);
      resetOscPhase();                    //Reset the oscillator
      
      executeTableEntry(2);               //Play pulse
      playWave(m);                        //Readout trigger
      waitWave();
    }}
    executeTableEntry(1);                 //Increment amplitude
    waitWave();
  }}
}} while (1);
"""

In [134]:
# Disable internal trigger
device.system.internaltrigger.enable(disable)

# Define command table
ct_schema = device.sgchannels[channel_index].awg.commandtable.load_validation_schema()
ct_rabi = CommandTable(ct_schema)
ct_rabi.table[0].waveform.playZero = True
ct_rabi.table[0].waveform.length = 32
ct_rabi.table[0].amplitude00.value = 0.0
ct_rabi.table[0].amplitude00.increment = False
ct_rabi.table[0].amplitude01.value = -0.0
ct_rabi.table[0].amplitude01.increment = False
ct_rabi.table[0].amplitude10.value = 0.0
ct_rabi.table[0].amplitude10.increment = False
ct_rabi.table[0].amplitude11.value = 0.0
ct_rabi.table[0].amplitude11.increment = False

ct_rabi.table[1].waveform.playZero = True
ct_rabi.table[1].waveform.length = 32
ct_rabi.table[1].amplitude00.value = 1/n_amps
ct_rabi.table[1].amplitude00.increment = True
ct_rabi.table[1].amplitude01.value = -1/n_amps
ct_rabi.table[1].amplitude01.increment = True
ct_rabi.table[1].amplitude10.value = 1/n_amps
ct_rabi.table[1].amplitude10.increment = True
ct_rabi.table[1].amplitude11.value = 1/n_amps
ct_rabi.table[1].amplitude11.increment = True

ct_rabi.table[2].waveform.index = 0


In [135]:
# Upload sequence
device.sgchannels[channel_index].awg.load_sequencer_program(seqc_rabi)

# Upload command table
device.sgchannels[channel_index].awg.commandtable.upload_to_device(ct_rabi)

# Enable sequencer
device.sgchannels[channel_index].awg.enable_sequencer(single = single)

In [136]:
device.system.internaltrigger.enable(enable)

# DRAG Pulse Calibration

In [137]:
seqc_drag = f"""
//Define constants
const num_qscale = {n_qscale};
const num_aves = {n_aves};
const amp = {amp};
const length = ceil(DEVICE_SAMPLE_RATE*{length}/32)*32;
const readout = ceil(DEVICE_SAMPLE_RATE*{readout}/32)*32;
const width = length/8;

//Waveform definition
wave wI = gauss(length,amp,length/2,width);
wave wQ = drag(length,amp,length/2,width);
wave m = marker(readout,1);

//Assign index and outputs
assignWaveIndex(1,2,wI,1,2,wQ,0);

//Execution, sequential averaging
do {{
  executeTableEntry(0);                       //Set start amplitude
  repeat (num_qscale-1) {{                    //Amplitude loop
    repeat (num_aves) {{                      //Averaging loop      
      waitDigTrigger(1);
      resetOscPhase();                        //Reset the oscillator
      
      executeTableEntry(2);                   //Play pulse
      playWave(m);                            //Readout trigger
      waitWave();
    }}
    executeTableEntry(1);                     //Increment amplitude
    waitWave();
  }}
}} while (1);
"""

In [138]:
# Disable internal trigger
device.system.internaltrigger.enable(disable)

device.sgchannels[channel_index].oscs[1].frequency = osc2_frequency
# Define command table
ct_schema = device.sgchannels[channel_index].awg.commandtable.load_validation_schema()
ct_drag = CommandTable(ct_schema)
ct_drag.table[0].waveform.playZero = True
ct_drag.table[0].waveform.length = 32
ct_drag.table[0].amplitude00.value = amp
ct_drag.table[0].amplitude00.increment = False
ct_drag.table[0].amplitude01.value = -q_start
ct_drag.table[0].amplitude01.increment = False
ct_drag.table[0].amplitude10.value = amp
ct_drag.table[0].amplitude10.increment = False
ct_drag.table[0].amplitude11.value = q_start
ct_drag.table[0].amplitude11.increment = False

ct_drag.table[1].waveform.playZero = True
ct_drag.table[1].waveform.length = 32
ct_drag.table[1].amplitude01.value = -q_step
ct_drag.table[1].amplitude01.increment = True
ct_drag.table[1].amplitude11.value = q_step
ct_drag.table[1].amplitude11.increment = True

ct_drag.table[2].waveform.index = 0

In [139]:
# Upload sequence
device.sgchannels[channel_index].awg.load_sequencer_program(seqc_drag)

# Upload command table
device.sgchannels[channel_index].awg.commandtable.upload_to_device(ct_drag)

# Enable sequencer
device.sgchannels[channel_index].awg.enable_sequencer(single = single)

In [140]:
device.system.internaltrigger.enable(enable)

# Rabi in a 3-level system

In [141]:
seqc_rabi = f"""
//Define constants
const num_amps = {n_amps};
const num_aves = {n_aves};
const amp = {amp};
const length = ceil(DEVICE_SAMPLE_RATE*{length}/32)*32;
const readout = ceil(DEVICE_SAMPLE_RATE*{readout}/32)*32;
const width = length/8;

//Waveform definition
wave wI = gauss(length,amp,length/2,width);
wave wQ = drag(length,amp,length/2,width);
wave m = marker(readout,1);

//Assign index and outputs
assignWaveIndex(1,2,wI,1,2,wQ,0);

//Execution, sequential averaging
do {{
  var amp = 0;
  repeat (num_amps-1) {{                      //Amplitude loop
    repeat (num_aves) {{                      //Averaging loop      
      waitDigTrigger(1);
      resetOscPhase();                        //Reset the oscillator
      
      executeTableEntry(num_amps);
      executeTableEntry(amp);                 //Play pulse
      
      playWave(m);                            //Readout trigger
      waitWave();
    }}
    amp += 1;
    waitWave();
  }}
}} while (1);
"""

In [142]:
# Disable internal trigger
device.system.internaltrigger.enable(disable)

device.sgchannels[channel_index].oscs[1].frequency = osc2_frequency
# Define command table
ct_rabi.clear()
ct_schema = device.sgchannels[channel_index].awg.commandtable.load_validation_schema()
ct_rabi = CommandTable(ct_schema)

for i in range(n_amps):
    ct_rabi.table[i].waveform.index = 0
    ct_rabi.table[i].amplitude00.value = i/n_amps
    ct_rabi.table[i].amplitude00.increment = False
    ct_rabi.table[i].amplitude01.value = 0
    ct_rabi.table[i].amplitude01.increment = False
    ct_rabi.table[i].amplitude10.value = i/n_amps
    ct_rabi.table[i].amplitude10.increment = False
    ct_rabi.table[i].amplitude11.value = 0
    ct_rabi.table[i].amplitude11.increment = False
    ct_rabi.table[i].oscillatorSelect.value = 1

ct_rabi.table[n_amps].waveform.index = 0
ct_rabi.table[n_amps].amplitude00.value = 0.6
ct_rabi.table[n_amps].amplitude00.increment = False
ct_rabi.table[n_amps].amplitude01.value = -0.6
ct_rabi.table[n_amps].amplitude01.increment = False
ct_rabi.table[n_amps].amplitude10.value = 0.6
ct_rabi.table[n_amps].amplitude10.increment = False
ct_rabi.table[n_amps].amplitude11.value = 0.6
ct_rabi.table[n_amps].amplitude11.increment = False
ct_rabi.table[n_amps].oscillatorSelect.value = 0

In [143]:
# Upload sequence
device.sgchannels[channel_index].awg.load_sequencer_program(seqc_rabi)

# Upload command table
device.sgchannels[channel_index].awg.commandtable.upload_to_device(ct_rabi)

# Enable sequencer
device.sgchannels[channel_index].awg.enable_sequencer(single = single)

In [144]:
device.system.internaltrigger.enable(enable)

# Randomized Benchmarking

In [78]:
# Define an envelope function for single qubit gates (Gaussian with width = length / 3)
def pulse_envelope(amplitude, length, phase, sigma=1/3, sample_rate=2.0e9, tol=15):
    #ensure waveform length is integer multiple of 16
    samples = round(sample_rate * length/ 16) * 16
    x = np.linspace(-1, 1, samples)
    # output is complex, so that phase determines the gate rotation axis
    y = amplitude * np.exp(-x**2 / sigma**2 + 1j * np.deg2rad(phase))

    return y.round(tol)

In [31]:
# All elements of the Clifford group, according to the definition in arXiv:1410.2338
clifford_params = [
    ['I'],
    ['Y/2', 'X/2'],
    ['-X/2', '-Y/2'],
    ['X'],
    ['-Y/2', '-X/2'],
    ['X/2', '-Y/2'],
    ['Y'],
    ['-Y/2', 'X/2'],
    ['X/2', 'Y/2'],
    ['X', 'Y'],
    ['Y/2', '-X/2'],
    ['-X/2', 'Y/2'],
    ['Y/2', 'X'],
    ['-X/2'],
    ['X/2', '-Y/2', '-X/2'],
    ['-Y/2'],
    ['X/2'],
    ['X/2', 'Y/2', 'X/2'],
    ['-Y/2', 'X'],
    ['X/2', 'Y'],
    ['X/2', '-Y/2', 'X/2'],
    ['Y/2'],
    ['-X/2', 'Y'],
    ['X/2', 'Y/2', '-X/2']
]

clifford_len = len(clifford_params)

# Parameters of basic single qubit pulses
pulses_params = {
    'I': {'amplitude':0.0, 'length': pi_length, 'phase': 0.0},
    'X': {'amplitude':pi_amplitude, 'length': pi_length, 'phase': 0.0},
    'Y': {'amplitude':pi_amplitude, 'length': pi_length, 'phase': 90.0},
    'X/2': {'amplitude':pi2_amplitude, 'length': pi2_length, 'phase': 0.0},
    'Y/2': {'amplitude':pi2_amplitude, 'length': pi2_length, 'phase': 90.0},
    '-X/2': {'amplitude':pi2_amplitude, 'length': pi2_length, 'phase': 0.0-180.0},
    '-Y/2': {'amplitude':pi2_amplitude, 'length': pi2_length, 'phase': 90.0-180.0},
}

# calculate complex waveforms for single qubit elementary pulses
pulses_waves = {pulse_type: pulse_envelope(**pulse_param) for (pulse_type, pulse_param) in pulses_params.items()}
# calculate complex waveforms for each of the Clifford gates
clifford_waves = [np.concatenate([pulses_waves[i] for i in clifford_gate]) for clifford_gate in clifford_params]

# divide real and complex part of waveforms into I and Q channel outputs
clifford_waves_real = [(np.real(wave), np.imag(wave)) for wave in clifford_waves]

In [145]:
# Waveform definition: allocating the waveform memory for the SeqC program
waveforms_def = ""
for i,wave in enumerate(clifford_waves):
    wave_len = len(wave)
    waveforms_def += f"assignWaveIndex(1,2,placeholder({wave_len}),1,2,placeholder({wave_len}),{i});"

# Define waveforms for waveform table
waveforms_rb = Waveforms()
for i in range(clifford_len):
    waveforms_rb[i] = (clifford_waves_real[i][0], clifford_waves_real[i][1])

In [146]:
seqc_rb = f"""
//Define constants
const clifford_len = {clifford_len};
const num_aves = {n_aves};
const readout = ceil(DEVICE_SAMPLE_RATE*{readout}/32)*32;

//Waveform definition
{waveforms_def}
wave m = marker(readout,1);

//Runtime parameters, set by user registers
//Sequence length
var m1 = getUserReg({awg_register_m1});
//PRNG seed
var seed = getUserReg({awg_register_seed});
//Recovery gate index
var recovery = getUserReg({awg_register_recovery});

//Configure the PRNG
setPRNGRange(0, clifford_len - 1);

//Execution, sequential averaging
while (1) {{
  var amp = 0;
  repeat (num_aves) {{                      //Averaging loop     
    // Initialize the PRNG
    setPRNGSeed(seed);
    
    waitDigTrigger(1);
    resetOscPhase();                        //Reset the oscillator
    
    // (Pseudo)-Random sequence of command table entries
    repeat (m1) {{
      var gate = getPRNGValue();
      executeTableEntry(gate);
    }}
    executeTableEntry(recovery);            //Final recovery gate
    
    playWave(m);                            //Readout trigger
    waitWave();
  }}
}}
"""

In [147]:
# Disable internal trigger
device.system.internaltrigger.enable(disable)

# Define command table
ct_schema = device.sgchannels[channel_index].awg.commandtable.load_validation_schema()
ct_rb = CommandTable(ct_schema)
for i in range(clifford_len):
    ct_rb.table[i].waveform.index = i

In [148]:
# Upload sequence
device.sgchannels[channel_index].awg.load_sequencer_program(seqc_rb)

# Upload waveforms
device.sgchannels[channel_index].awg.write_to_waveform_memory(waveforms_rb)

# Upload command table
device.sgchannels[channel_index].awg.commandtable.upload_to_device(ct_rb)

# Enable sequencer
device.sgchannels[channel_index].awg.enable_sequencer(single = single)

In [152]:
# Define number of seeds and gates. Try playing around with these values!
m1             = 10667
seed           = 123321431
recovery       = 3

# Set user register values
with device.set_transaction():
    device.sgchannels[channel_index].awg.userregs[awg_register_m1](m1)
    device.sgchannels[channel_index].awg.userregs[awg_register_seed](seed)
    device.sgchannels[channel_index].awg.userregs[awg_register_recovery](recovery)

# Enable sequencer
device.sgchannels[channel_index].awg.enable(disable)
device.sgchannels[channel_index].awg.enable_sequencer(single = single)

In [150]:
device.system.internaltrigger.enable(enable)

# Turn of Internal Trigger

In [117]:
with device.set_transaction():
    device.system.internaltrigger.enable(disable)
    device.sgchannels[channel_index].awg.enable(disable)